In [1]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# # ---- 데이터 구경 --- 

# for (X_train, y_train) in dataset['train_loader'] :
#     for idx in range(100,150):
#         if 1 in y_train[idx]:
#             plt.plot(X_train[idx][0])
#             plt.title(y_train[idx])
#             plt.show()
#     break

In [2]:
from _preprocessing import *

params = dict()

# ------------------ configuration ------------------

params['source']                 = 'STEAD'
params['mode']                   = 'generator'
params['window_count']           = 6
params['condition']              = 'M>3'
params['imbalance']              = False
params['config_name']            = make_config(params)
params['file_name']              = params['source'] + '_' + params['config_name'] + '_dump.pkl'
params['model_name']             = 'CNN_' + params['config_name']
params['output']                 = 'lstm'

# ------------------ preprocessing ------------------

params['output_name']            = 'test_trainer'
params['input_dimention']        = (6000, 3)
params['shuffle']                = True
params['label_type']             = 'gaussian'
params['normalization_mode']     = 'std'
params['augmentation']           = True
params['add_event_r']            = 0.6
params['add_gap_r']              = 0.2
params['add_noise_r']            = 0.5
params['drop_channel_r']         = 0.5
params['shift_event_r']          = 0.9
params['scale_amplitude_r']      = None
params['pre_emphasis']           = False

params['batch_size']             = 36
params['total_data_size']        = 20000
params['T/F_ratio']              = [0.5, 0.5]
params['train_valid_test_split'] = [0.60, 0.20, 0.20]

# ----------------- model-parameter -----------------

params['kernel_size']            = 10

# ----------------- train-parameter -----------------

params['epochs']                 = 10
params['lr']                     = 0.001

import pprint
pprint.pprint(params)

preprocessing_generator(params)

Using TensorFlow backend.


{'T/F_ratio': [0.5, 0.5],
 'add_event_r': 0.6,
 'add_gap_r': 0.2,
 'add_noise_r': 0.5,
 'augmentation': True,
 'batch_size': 36,
 'condition': 'M>3',
 'config_name': 'W6_M>3_balance',
 'drop_channel_r': 0.5,
 'epochs': 10,
 'file_name': 'STEAD_W6_M>3_balance_dump.pkl',
 'imbalance': False,
 'input_dimention': (6000, 3),
 'kernel_size': 10,
 'label_type': 'gaussian',
 'lr': 0.001,
 'mode': 'generator',
 'model_name': 'CNN_W6_M>3_balance',
 'normalization_mode': 'std',
 'output': 'lstm',
 'output_name': 'test_trainer',
 'pre_emphasis': False,
 'scale_amplitude_r': None,
 'shift_event_r': 0.9,
 'shuffle': True,
 'source': 'STEAD',
 'total_data_size': 20000,
 'train_valid_test_split': [0.6, 0.2, 0.2],
 'window_count': 6}
file exist!!!


In [3]:
from _utils import *

dataset = load_dataset(params)

DEVICE 		:  cuda:1
pid 		:  3176

---------------------------------Data---------------------------------

file name 	: STEAD_W6_M>3_balance_dump.pkl
data length 	: 20000
T/F count 	: 10000 / 10000
T/F ratio 	: 50.0 %
Train size 	: 12000
Valid size 	: 4000
Test size 	: 4000

------------------------------Data Loader------------------------------

X_train shape:  torch.Size([500, 3, 6000])  	type :  torch.FloatTensor
y_train shape:  torch.Size([500, 1, 6])  	type :  torch.FloatTensor


In [4]:
from _CNN_Window6 import *

model = train(params, dataset)

 Epoch [1/10], Step [24/24], Loss: 1.4372, Val Loss: 2.1287, accuracy : 0.6060, precision : 0.9792, recall : 0.1934, F1_score : 0.3230, TP : 47, FP : 1, TN : 256, FN : 196
 Epoch [2/10], Step [24/24], Loss: 0.7761, Val Loss: 1.1386, accuracy : 0.7980, precision : 0.9649, recall : 0.6346, F1_score : 0.7657, TP : 165, FP : 6, TN : 234, FN : 952
 Epoch [3/10], Step [24/24], Loss: 0.7923, Val Loss: 0.9782, accuracy : 0.8140, precision : 0.9845, recall : 0.6786, F1_score : 0.8034, TP : 190, FP : 3, TN : 217, FN : 9011
 Epoch [4/10], Step [24/24], Loss: 0.6607, Val Loss: 0.8566, accuracy : 0.8520, precision : 0.9511, recall : 0.7292, F1_score : 0.8255, TP : 175, FP : 9, TN : 251, FN : 658
 Epoch [5/10], Step [24/24], Loss: 0.6957, Val Loss: 0.8676, accuracy : 0.8580, precision : 0.9619, recall : 0.7623, F1_score : 0.8505, TP : 202, FP : 8, TN : 227, FN : 635
 Epoch [6/10], Step [24/24], Loss: 0.6491, Val Loss: 0.9074, accuracy : 0.8200, precision : 0.9126, recall : 0.7231, F1_score : 0.8069,

In [9]:
test(model, dataset)

Loss: 0.6553, accuracy : 0.8740, precision : 0.9420, recall : 0.7927, F1_score : 0.8609
TP : 195, FP : 12, TN : 242, FN : 51


In [6]:
import torchsummary

torchsummary.summary(model, (3, 6000), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 4, 6001]             124
              ReLU-2              [-1, 4, 6001]               0
           Dropout-3              [-1, 4, 6001]               0
         MaxPool1d-4              [-1, 4, 3000]               0
            Conv1d-5              [-1, 8, 3001]             264
              ReLU-6              [-1, 8, 3001]               0
           Dropout-7              [-1, 8, 3001]               0
         MaxPool1d-8              [-1, 8, 1500]               0
            Linear-9                    [-1, 1]           2,001
          Sigmoid-10                    [-1, 1]               0
Total params: 2,389
Trainable params: 2,389
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.07
Forward/backward pass size (MB): 1.28
Params size (MB): 0.01
Estimated Total